In [1]:
!pip install --upgrade openai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client plyer

In [11]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os, pickle

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.pkl', 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)

In [12]:

import os
import pickle
import base64
import openai
from email.mime.text import MIMEText
from dotenv import load_dotenv
from openai import OpenAI
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
if os.path.exists("token.pkl"):
    os.remove("token.pkl")

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
service = authenticate_gmail()



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=295471913887-v68iukod7s4c2oqusgh056vk0iqml9gr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55386%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=w9KlSRCiAfONfzikklX6xSA0aN6hIN&access_type=offline


In [13]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [14]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os, pickle

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def authenticate_gmail():
    creds = None
    if os.path.exists('token.pkl'):
        with open('token.pkl', 'rb') as token:
            creds = pickle.load(token)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.pkl', 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)

In [15]:
def get_emails(service, max_results=5):
    results = service.users().messages().list(userId='me', labelIds=['INBOX'], maxResults=max_results).execute()
    messages = results.get('messages', [])
    
    email_data = []
    for msg in messages:
        msg_detail = service.users().messages().get(userId='me', id=msg['id'], format='full').execute()
        headers = msg_detail['payload']['headers']
        
        email_info = {
            'snippet': msg_detail.get('snippet', ''),
            'to': next((h['value'] for h in headers if h['name'] == 'From'), None),
            'subject': next((h['value'] for h in headers if h['name'] == 'Subject'), ''),
            'msg_id': next((h['value'] for h in headers if h['name'] == 'Message-ID'), None),
        }
        email_data.append(email_info)
    
    return email_data

In [16]:

def classify_and_respond(email_text):
    prompt = f"""You are an intelligent email assistant. 
Classify the following email as one of: urgent, spam, or follow-up.
Then generate a short, polite response.

Email: {email_text}

Respond in this format:
Classification: <category>
Reply: <your reply>
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )

    result = response.choices[0].message.content
    classification = None
    reply = None

    for line in result.splitlines():
        if line.lower().startswith("classification:"):
            classification = line.split(":", 1)[1].strip().lower()
        elif line.lower().startswith("reply:"):
            reply = line.split(":", 1)[1].strip()

    return classification, reply


In [17]:


from email.mime.text import MIMEText
import base64

def create_draft(service, to, subject, reply_text, in_reply_to=None):
    message = MIMEText(reply_text)
    message['to'] = to
    message['subject'] = "Re: " + subject
    if in_reply_to:
        message['In-Reply-To'] = in_reply_to

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    draft_body = {'message': {'raw': raw}}
    
    draft = service.users().drafts().create(userId='me', body=draft_body).execute()
    print(f" Draft created for: {to}")



In [18]:
service = authenticate_gmail()
emails = get_emails(service)

for email in emails:
    classification, reply = classify_and_respond(email.get("snippet", ""))

    print("---------")
    print("Original:", email.get("snippet", ""))
    print("Classification:", classification)
    print("Reply:", reply)

    if classification != "spam" and reply:
        create_draft(
            service,
            to=email.get("to"),
            subject=email.get("subject"),
            reply_text=reply,
            in_reply_to=email.get("msg_id")
        )
    else:
        print(" Skipped (spam or no reply)")

---------
Original: Hii, How are you doing ?
Classification: follow-up
Reply: Hello! I'm doing well, thank you for asking. How can I assist you today?
 Draft created for: Alex <temporaryuse1871@gmail.com>
---------
Original: You allowed Credentials access to some of your Google Account data amarnathreddy1871@gmail.com If you didn&#39;t allow Credentials access to some of your Google Account data, someone else may be trying
Classification: urgent
Reply: Thank you for bringing this to my attention. I did not authorize this access. Please guide me on how to secure my account and prevent any unauthorized access.
 Draft created for: Google <no-reply@accounts.google.com>
---------
Original: A new sign-in on Windows amarnathreddy1871@gmail.com We noticed a new sign-in to your Google Account on a Windows device. If this was you, you don&#39;t need to do anything. If not, we&#39;ll help you
Classification: follow-up
Reply: Thank you for the notification. I will review the sign-in activity to en